## About this notebook
This jupyter notebook can be run on any computer with a standard browser and no prior installation of any programming language is required.

### About

Stain normalization is a common preprocessing step that aims to reduce color differences to a minimum that are not important in clinical practice.
In digital pathology, in order to have better results with different scanners and different types of colors, normalization is used to reduce errors.

### Setting up :
You should install our package and import to your code , then use it .


1. At the first you should import some packages that you need .

In [ ]:
import os
from readwsi.readwsi import ReadWsi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from readwsi.normalization import Normalization
import readwsi.normalization as normalization
from PIL import Image
mpl.rcParams["figure.dpi"] = 150 

### Reading in a WSI
It's easy to read a WSI. 

In [ ]:
# get wsi directory and wsi name of user


while True:
    
    # loading input directory
    dir_file = input(r"enter the file directory ...?    forexample: C:\Users\data ---->")
    
    # Getting the wsi file name
    file_name = input("What is the file name...?    forexample: a.svs ---->")
    
    print("\n")
    path = dir_file + "\\" + file_name
    
    #Checking folder availability
    if os.path.exists(path):
        wsi_obj =ReadWsi.wsi_reader(path)
        break
    else:
        print(" !!! path file error......." )

print(" Done...")

In [ ]:
# # set up  wsi directory  and  wsi name

# dir_file = r"C:\Users\data"                       
# file_name = 'CMU-2.svs'

# # use of ReadWsi class for reading wsi file

# wsi_obj =ReadWsi.wsi_reader(dir_file+'/' + file_name)

# print(wsi_obj)

### print important metadata.

In [ ]:
wsi_info = wsi_obj.level_dimensions

#A list of (width, height) tuples, one for each level of the slide. level_dimensions[k] are the dimensions of level k"
print("size of ever dimensions are : ",wsi_info)
print("A (width, height) tuple for level 0 of the slide : ",wsi_obj.dimensions)


#The number of levels in the slide. Levels are numbered from  0   (highest resolution)  to  level_count  -1  (lowest resolution)")
print("The number of levels in the slide is : ",wsi_obj.level_count)


#A list of downsample factors for each level of the slide. level_downsamples[k] is the downsample factor of level k.
print("A list of downsample factors for each level of the slide is : ",wsi_obj.level_downsamples)


### plot the file :

In [ ]:
# plot the wsi with size of ane of level dimensions 



#### read_region(location, level, size)

#location (tuple) = tuple giving the top left pixel in a level of wsi (0 or 1 or ...)
#level (int) =  The number of levels in the slide. Levels are numbered from 0 (highest resolution) to level_count - 1 (lowest resolution).
# size (tuple) – (width, height) tuple giving the region size

location = (0,0)                              # location (tuple) – (x, y)
level = 1                                     # level(int)
size = wsi_obj.level_dimensions[1]            # a tuple of a level of the slide that here   level dimensions  is 1                  
    
    
file = wsi_obj.read_region(location,level,size).convert('RGB')
plt.imshow(file)

### Extract a tile from the WSI

In [ ]:
while True:
    # location (tuple) – (x, y)
    location = [int (input("please enter x-coordinate ... ")),int (input("please enter y-coordinate ..."))]
    # level(int)           
    level = int(input(r"please enter The number of levels in the slide..."))
    # a tuple of a level of the slide that here   level dimensions  is 1 
    size_ =int( input(r"please enter size of tile... "))
    size= [size_,size_]
    break
    
print ("\n location is ",location, "  - level is ",level," - size is ", size)
sample = wsi_obj.read_region( location, level, size)

plt.figure(figsize=(3, 3))
plt.imshow(sample)

### Stain normalizing a tile

In [ ]:


#rgba_image = Image.open('tile path')
# Convert RGBA to RGB (it should have 3 channels)
rgb_image = sample.convert('RGB')

temp_tile_np = np.array(rgb_image)
print("size of patch =" , temp_tile_np.shape)


# normalization the tile 
norm_img, H_img, E_img = normalization.norm_HnE(temp_tile_np, Io=240, alpha=1, beta=0.15)
    


### plot the tile  with  Normalized Stain ،  H tile  and  E tile

In [ ]:

row =1
col =4
axes=[]
fig=plt.figure()  
plt.rcParams.update({'font.size': 8})



fig = plt.figure(figsize=(15, 15))

axes.append( fig.add_subplot(row, col, 1) )
subplot_title=("original_img ")
axes[-1].set_title(subplot_title)  
plt.imshow(rgb_image)

axes.append( fig.add_subplot(row, col, 2) )
subplot_title=("norm_img")               
axes[-1].set_title(subplot_title)  
plt.imshow(norm_img)
               
axes.append( fig.add_subplot(row, col, 3) )
subplot_title=("H_img")              
axes[-1].set_title(subplot_title)  
plt.imshow(H_img)
               
axes.append( fig.add_subplot(row, col, 4) )
subplot_title=("E_img")
axes[-1].set_title(subplot_title)  
plt.imshow(E_img)


### visualization histogram 

In [ ]:
# type of input images are numpy.ndarray

visualize = True

source_matched, source, target= Normalization.match_hist(H_img, E_img , visualize)

### patch normalization 
##### This function perform patch normalization for all patches inside subfolders.

In [ ]:
while True:
    # loading input directory
    input_dir = input(r"please enter the patches folder path ...forexample: C:\Users\data\  ---->")
    # loading output directory
    output_dir =input(r"please enter the output folder path ... forexample: C:\Users\data\norm_ ---->")
    # # loading file extension
    file_extension = input(r"please enter file extension...    forexample: tif  ---->")
   
    
    #Checking folder availability 
    if os.path.exists(input_dir):
        Normalization.patch_norm(input_dir , output_dir , 'H_E' , file_extension)
        break
    else:
        print("path not exist..try again")
        continue

## normalization all patches  and  plot some patches

In [ ]:
# patch normalization  and  get patch directory and out directory of user 
import os

while True:

    input_dir = input("enter the patches directory ...   forexample: C:\\Users\\data ----> ")
    
    if os.path.exists(input_dir):
        print("ok") 
    else :
        print(" !!! input_dir error....... please enter carefully" )
        continue
        
     
    output_dir = input("enter the output directory ...   forexample: C:\\Users\\data\\out  ----> ")
    ''' 
    if os.path.exists(output_dir):
        print("ok") 
    else:
        print(" !!! output_dir error....... please enter carefully" )
        continue
    ''' 
    file_extension =input("What is the file extension ...    forexample: tif ----> ")
    

    if (file_extension == 'tif' or file_extension == 'png' or file_extension == 'jpg'):
        print("ok") 
        break
    else:
        print(" !!! file_extension error....... please enter carefully" )
        continue     


print("\n Done...")

#  If the value of the  "vis"  is True = The output is also displayed as a plot 
vis = True

Normalization.patch_norm(input_dir , output_dir , 'H_E' , file_extension,vis)

## use of patch extraction with normalization function

In [ ]:
while True:

    
    patchsize = int(input(r"please enter patch size ...    forexample: 255 ---->"))
    
    output_directory = input(r"please enter output directory ...    forexample: C:\Users\data\output ---->")
    
    
    # loading random state , 1 is True , 0 is False . if 1 entered , patches are extarct Randomly from the whole slide image
    # if 0 entered patches extract From the beginning of the image with a loop on the rows and columns
    # Any part that contains an image, not a white page, is extracted
    
    random_state = int(input(r"please enter random state ...    1 = randomize ,  2 = non-randomized  ---->"))
    
    if random_state==1 :
        random_state= True
    if random_state==2 :
        random_state= False
        
    patch_numbers = int(input(r"please enter patch numbers...  ---->"))
    #  If the value of the  "vis"  is 1 = The output is also displayed as a plot
    vis_state = int(input("do you want to plot some patches ? (1 = yes i want  ,   2 = no  ) ---->"))
    
    print("\n")
    
    #Checking folder availability
    if os.path.exists(output_directory):
        if random_state==True or random_state==False:
            ReadWsi.patch_extraction_with_normalized_tiles(wsi_obj,patchsize,output_directory,random_state,patch_numbers)
            break
        else :
            print('random_state that entered is wrong')
            continue
    else:
        print('output_directory that entered is wrong')
        continue

In [ ]:
patchsize= 256

output_directory = dir_file + '\\out_dir\\'

print("\n**********************< without randam state >************************\n")


    # loading random state , 1 is True , 0 is False . if 1 entered , patches are extarct Randomly from the whole slide image
    # if 0 entered patches extract From the beginning of the image with a loop on the rows and columns
    # Any part that contains an image, not a white page, is extracted
    
random_state = False
# Number of patches to be extracted
patch_numbers = 2

ReadWsi.patch_extraction_with_normalized_tiles(wsi_obj,patchsize,output_directory,random_state,patch_numbers)


print("\n**********************< with randam state >************************\n")
#  If the value of the random_state is True = patches are randomly extracted
random_state = True
patch_numbers = 2

ReadWsi.patch_extraction_with_normalized_tiles(wsi_obj,patchsize,output_directory,random_state,patch_numbers)


## Reinhard Normalization

#Reinhard is a method for color normalization whole slide images

In [ ]:
image = input ("Enter image path : ")


image , image_norm = Normalization.reinhard(image)

fig, axarr = plt.subplots(1,2)
axarr[0].imshow(image)
axarr[0].set_title("input image")
axarr[1].imshow(image_norm)
axarr[1].set_title("Reinhard normalization")